#Translation - T5

####Tento notebook se zabývá převodem textu z jednoho jazyka do druhého na základě námi vytrénovaného nebo již předtrénovaného modelu T5 s využitím datasetu, na kterém byl model natrénovaný.

*Knihovna T5 slouží k načítání, předzpracování, kombinování a evaluaci datových sad. Poskytuje také možnost dotrénovat (fine-tune) předtrénované modely.*

*Knihovna může být využita pro budoucí vývoj modelů poskytováním užitečných modulů pro trénování a dotrénování (potenciálně rozsáhlých) modelů na směsici text-to-text úloh.*

- Odkaz na využívaný repozitář:
[Repozitář](https://github.com/google-research/text-to-text-transfer-transformer.git)

- Návod Hugging Face: [HuggingFace](https://huggingface.co/docs/transformers/tasks/translation?fbclid=IwAR13_un46roXn_ggkjeiqQFovRYmKVyeOpGFLDstlGRYZHzm14CQqpCnnJ4)

### 1. Instalace potřebných packages

*   **transformers** - knihovna od HuggingFace poskytuje přístup k předtrénovaným modelům pro zpracování přirozeného jazyka, včetně modelů z rodiny Transformer, jako je např. T5, BERT nebo GPT
*   **datasets** - poskytuje přístup k mnoha datasetům pro zpracování přirozeného jazyka, obsahuje funkce pro načítání, předzpracování a práci s různými datovými sadami, což zjednodušuje experimentování s různými daty
*   **evaluate** - pro vyhodnocování výsledků modelu, v tomto kontextu obsahuje kód pro vyhodnocování kvality překladů nebo výkonu modelu na konkrétní úloze
*   **sacrebleu** - používán pro vyhodnocování přesnosti překladů v kontextu práce s překladatelskými modely





In [1]:
! pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00


##Zobrazení obsahu datasetu

###Funkce *load_dataset* slouží k načtení datové sady ***opus_books*** v jazykovém páru angličtina–francouzština ("en-fr").



In [2]:
from datasets import load_dataset #načítání různých datových sad pro práci se zpracováním přirozeného jazyka

books = load_dataset("opus_books", "en-fr") #načtení datové sady "opus_books" pro překlad z angličtiny do francouzštiny

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

Rozdělení trénovacích dat (uložených v *books["train"]*) na dvě části: novou trénovací část a testovací část. Tato práce s testovací množinou je užitečná při evaluaci modelu, protože umožňuje ověřit jeho schopnost generalizace na neznámá data.

In [3]:
books = books["train"].train_test_split(test_size=0.2) #rozdělení datové sady na trénovací a testovací množiny - argument testsize=0.2 = 20% train a 20% test

In [4]:
books["train"][0]

{'id': '119099',
 'translation': {'en': 'From the Pré-des-Vaches it was not two kilometres to the point of Herbes-Rousses, in five strokes.',
  'fr': "Du Pré-des-Vaches, il n'y avait pas deux kilometres a la pointe des Herbes-Rousses: en cinq coups."}}

####Přístup k předtrénovanému tokenizeru pro model T5:
Třída *AutoTokenizer* slouží k načtení předtrénovaného tokenizeru pro model T5.
Získáme přístup k předtrénovanému tokenizeru pro model T5 (t5-small). Tento tokenizer můžeme následně používat k převodu textu na tokeny - obvyklý krok před vstupem do modelu.







In [6]:
from transformers import AutoTokenizer

checkpoint = "t5-small" # checkpoint = předtrénovaný model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

####Předzpracování dat

In [7]:
source_lang = "en" #zdrojový jazyk
target_lang = "fr" #cílový jazyk
prefix = "translate English to French: "

#tato funkce přijímá slovník příkladů (např., trénovací nebo testovací data) a vrací připravená data pro model
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] #seznam vstupů pro model-pro každý příklad v datovém slovníku přidává prefix a kombinuje ji s anglickým textem
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True) #tokenizer-převod vstupních a cílových textů na tokeny
    # max_length=128,truncation=True = omezení délky vstupu na max. 128 tokenů a provedení oříznutí textu - pokud přesáhne tuto délku
    return model_inputs

###*Po provední této operace bude každý příklad obsahovat tokeny vstupů a cílů, které jsou pak připravené pro vstup do modelu.*

In [8]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

**Funkce preprocess** předzpracovává jednotlivé příklady datové sady - pro každý příklad vytváří vstup pro model pomocí tokenizeru. Výsledek funkce obsahuje tokeny vstupů a cílů pro každý příklad

**Metoda map** aplikuje funkci preprocess_function na každý prvek datové sady.

**Parametr batched=True** - funkce preprocess_function bude aplikována na dávky dat najednou, je to efektivní při práci s velkými datovými sadami

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint, return_tensors="tf")
#return_tensors="tf" - výstupní data mají být ve formátu TensorFlow tenzorů -> TensorFlow knihovna

Třída **DataCollatorForSeq2Seq** slouží k vytváření objektu, který dokáže zpracovat data pro trénování nebo vyhodnocování modelů.

##**Evaluate (vyhodnocení)**

Evaluate = vyhodnocování - proces posuzování výkonu modelu na testovacích datech.

In [10]:
import evaluate

metric = evaluate.load("sacrebleu") #načtení metriky s názvem sacrebleu -> hodnocení kvality strojových překladů

Funkce **postprocess_text** a **compute_metrics** slouží k postprocessingu (proces úprav nebo manipulace s daty) a výpočtu metrik na výsledcích vyhodnocování modelu, zejména v kontextu překladu textu.

In [11]:
import numpy as np #knihovna pro práci s vícerozměrnými poli (např. matice nebo tenzory)


def postprocess_text(preds, labels):            #funkce přijímá predikce (preds) a skutečné hodnoty (labels)
    preds = [pred.strip() for pred in preds]          #odstranění mezer na začátku a konci -> výstup ve formátu seznamu
                                                      #preds = odhady nebo generovaný text, který model vytvořil v odpovědi na vstupní text
    labels = [[label.strip()] for label in labels]    #labels = překlady ve skutečném cílovém jazyce

    return preds, labels


def compute_metrics(eval_preds):        #přijímá vyhodnocené předpovědi (eval_preds) - obsahují preds a labels
    preds, labels = eval_preds          #rozbalení vyhodnocených předpovědí
    if isinstance(preds, tuple):        #ověření, zda jsou preds ve formě seznamu, pokud ano, vezme se první element a slouží k zajištění toho, že preds obsahuje hlavní výstupy modelu, které budou následně dekódovány a použity pro výpočet metrik
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #dekódování modelových výstupů z tokenů->text pomocí tokenizeru; speciální tokeny jsou přeskakovány (skip_special_tokens=True).

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #nahrazení hodnoty -100 v labels tokenem pro vyplnění (pad token) z tokenizeru.
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True) #dekódování skutečných hodnot (labels) z tokenů -> text

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #funkce postprocess_text pro další úpravy dekódovaných preds a labels

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #výpočet metrik na základě dekódovaných preds a labels
    result = {"bleu": result["score"]} #uložení výsledku metriky BLEU do result

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds] #výpočet délky generovaných předpovědí bez použití tokenů pro vyplnění (pad token)
    result["gen_len"] = np.mean(prediction_lens) # uložení průměrné délky generovaných předpovědí do slovníku result
    result = {k: round(v, 4) for k, v in result.items()} #zaokrouhlování výsledných hodnot na čtyři desetinná místa
    return result

##**Příklad dotrénování modelu (fine-tune)**

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

**Learning_rate** (rychlost učení) určuje jak rychle se model učí během tréninku a jak velké kroky dělá při aktualizaci svých váh.

**Weight_decay_rate** (váhový útlum) zabrání tomu, aby se váhy modelu potenciálně přizpůsobily trénovacím datům až příliš.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(checkpoint) #checkpoint = název předtrénovaného modelu

 **Třída TFAutoModelForSeq2SeqLM** představuje architekturu modelu pro sekvenční generaci v našem případě pro překlad.

 **Metoda from_pretrained** načítá váhy (weights) a konfiguraci modelu z předtrénované verze.

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_books["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = model.prepare_tf_dataset(
    tokenized_books["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

**Metoda model.prepare_tf_dataset** připravuje TensorFlow dataset, bere tokenizovaná data (v tomto případě ze slovníku tokenized_books pro trénovací a testovací množinu), specifikuje různé parametry, jako je míchání (shuffle), velikost dávky (batch_size) a funkce pro zpracování dat (collate_fn).

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

**Metoda compile** slouží pro kompilaci modelu v TensorFlow, obvykle se používá v kontextu tréninku modelu a slouží k nastavení několika důležitých parametrů, včetně optimalizačního algoritmu.

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

 Tato instance třídy slouží k monitorování metrik během tréninku modelu. Během každé epochy, po skončení tréninku na daném batchi.

**Callback funkce** vypočítá metriky pomocí zadané funkce compute_metrics a evaluačního datasetu tf_validation_set.

Takové zpětné volání může být užitečné při tréninku modelu, abychom mohli sledovat jeho výkonnost během procesu učení a případně přizpůsobit tréninkové parametry.

#### **Nahrání modelu na HuggingFace*

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="my__model",
    tokenizer=tokenizer,
)

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

###Spuštění tréninku

Metoda **fit** provede trénink modelu na trénovacích datech (tf_train_set) po určený počet epoch a vyhodnotí model na testovacích datech (tf_test_set) po každé doběhnuté epoše.

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3, callbacks=callbacks)

##**Otestování funkčnosti předtrénovaného modelu T5-base**

Připravíme si vstupní text, tokenizujeme ho, použijeme model pro generaci překladu a následně dekódujeme výsledek zpět do lidsky čitelné podoby.

**1. Překlad z angličtiny do němčiny:**

In [15]:
en_text = input("translate English to German: ")

print("Enter the text you want to translate:", en_text)


translate English to German: Hello I am Johnny and what´s your name
Enter the text you want to translate: Hello I am Johnny and what´s your name


Nástroj **pipeline** umožňuje jednoduchý přístup k předtrénovaným modelům pro různé úlohy, v našem případě pro překlad.

In [16]:
from transformers import pipeline
translator = pipeline("translation", model="t5-base")
input_text = f"translate English to German: {en_text}"
result = translator(input_text)
output = {'inputs': input_text, 'targets': result[0]['translation_text']}

print(output)

{'inputs': 'translate English to German: Hello I am Johnny and what´s your name', 'targets': 'Hallo, ich bin Johnny und was ist Ihr Name?'}


In [19]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")
inputs = tokenizer(en_text, return_tensors="tf").input_ids

In [23]:
inputs

<tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[ 8774,    27,   183, 18398,    11,   125,     3,     2,     7,
           39,   564,     1]], dtype=int32)>

Vytvoření tokenizeru pro model "t5-base" a používá se k tokenizaci anglického textu (en_text). Identifikátory tokenů jsou uloženy v proměnné inputs.

Vygenerujeme sekvenci tokenů pomocí modelu pro sekvenční generaci "t5-base" na základě zadaných identifikátorů tokenů (inputs). Výsledek je uložen v proměnné outputs.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-base")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [22]:
outputs #identifikátory tokenů

<tf.Tensor: shape=(1, 41), dtype=int32, numpy=
array([[    0, 32099, 18398,    11,     3,    23,   183, 18398,     5,
          499,   564,    19, 18398,    11,   125,     3,     2,     7,
           39,   564,     3,     2,     7,    39,   564,    19, 18398,
           58,  8774, 18398,     5,    27,   183, 18398,    11,   125,
            3,     2,     7,    39,   564]], dtype=int32)>


Dekódování výstupu modelu zpět do čitelného textu:

In [21]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'Johnny and i am Johnny. My name is Johnny and what s your name s your name is Johnny? Hello Johnny. I am Johnny and what s your name'